In [1]:
import numpy as np
import pandas as pd

In [2]:
num_data = pd.read_csv("data.csv")

In [3]:
target = "Class_b\'2\'"
pos = num_data[target].mean()
neg = 1 - pos

# Train-test split

In [4]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

from keras import models
from keras import layers
from keras import optimizers 
from keras import metrics

In [5]:
## this function is task-specific
def model_train(model, split, epochs, random_target=False):
    model.compile(optimizer=optimizers.Adam(), 
              loss='binary_crossentropy', 
              ) 
    
    x_train, x_test, y_train, y_test = split

    if random_target:
        y_train = np.random.randint(2, size = len(y_train))
        y_test = np.random.randint(2, size = len(y_test))  

    # batch_size made bigger to ensure some important cases are included    
    history = model.fit(x_train
                        , y_train
                        , epochs=epochs
                        , batch_size=min(32, len(x_train))
                        , validation_data=(x_test, y_test)
                        # , class_weight=class_weight
                    )
    return history, model

In [6]:
def sample(data, cut, target):
    assert(cut > 0)
    return data.groupby(target, group_keys=False).apply(lambda x: x.sample(frac=1/cut))    

In [7]:
def get_train_test_split(data, target, cut, random_state = None):
    num_data_new = sample(data, cut, target)
    
    train, test = train_test_split(num_data_new, test_size=0.2, random_state=random_state) 

    y_train = np.array(train[target])
    y_test = np.array(test[target])

    x_train = np.array(train.drop(columns=[target]))
    x_test = np.array(test.drop(columns=[target]))
    
    return x_train, x_test, y_train, y_test 

# Creating models

In [39]:
params = [15, 32, 32, 1]

In [40]:
input_size = len(num_data.columns) - 1

In [41]:
input_size

14

In [42]:
def make_model(input_size, parameters):
    model = models.Sequential()

    model.add(layers.Dense(parameters[0], activation='relu', input_shape=(input_size,))) # , kernel_regularizer="l2"
    for i in range(1, len(parameters)-1):
        model.add(layers.Dense(parameters[i], activation="relu")) #, kernel_regularizer="l1")
    model.add(layers.Dense(parameters[-1], activation='sigmoid')) # , kernel_regularizer="l1"
    
    model.compile(optimizer=optimizers.Adam(), 
              loss='binary_crossentropy', 
              metrics=["accuracy"]
              )
    
    return model

In [43]:
volume = len(num_data)

In [44]:
cuts = [1, 2, 4, 8, 16, 32, 64, 128, 254, 1024, 2048]

In [45]:
print("Cuts are {}".format(cuts))
print("Volume of data is {}".format(list(map(lambda x: int(volume / x), cuts))))

Cuts are [1, 2, 4, 8, 16, 32, 64, 128, 254, 1024, 2048]
Volume of data is [14980, 7490, 3745, 1872, 936, 468, 234, 117, 58, 14, 7]


In [46]:
get_model_directory = lambda series, cut: "models/{}_series/{}_cut/".format(series, cut)
get_model_name = lambda number:  "m_{}.h5".format(number)

# Logging

In [47]:
import datetime 
def log_preamble(log_file_name):
    global tries, epochs, series, file_name
    time_stamp = datetime.datetime.now()

    with open(log_file_name, "a") as log_file:
        log_file.write(f"\n\nExecuted on time is {datetime.datetime.now()}\n")
        log_file.write(f"Tries: {tries}, epochs: {epochs}, series = {series}\n")
        log_file.write(f"Network configuration is {params}\n")
    return time_stamp

def log_final(log_file_name, time_stamp):
    time_stamp_new = datetime.datetime.now()
    with open(log_file_name, "a") as log_file:
        log_file.write(f"Finished successfully at {time_stamp_new}\n")
        log_file.write(f"Total time = {(time_stamp_new - time_stamp).total_seconds()}\n")

In [48]:
model_name = lambda series_num, cut_val, model_num: get_model_directory(series_num, cut_val) + get_model_name(model_num)

# Training models

In [49]:
from keras.callbacks import ModelCheckpoint
# В теле этой функции задаются параметры обучения, чекпоинты
def model_train(data_split, batch_size, epochs, random_target=False):
    model = make_model(input_size, params)

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        verbose=1,
        patience=15,
        mode='auto',
        restore_best_weights=True)

    x_train, x_test, y_train, y_test = data_split

    if random_target:
        y_train = np.random.randint(2, size = len(y_train))
        y_test = np.random.randint(2, size = len(y_test))  

    history = model.fit(x_train, y_train,
                    epochs=epochs,
                    batch_size=min(batch_size, len(x_train)),  
                    validation_data=(x_test, y_test), 
                    callbacks = [early_stopping]) 
    return history, model

In [50]:
series = [4]
tries = 20 
epochs = 128
batch_size = 512
histories = [[[] for _ in cuts] for _ in series] 

log_file = "model_log.txt"
t = log_preamble(log_file)

for i, s in enumerate(series):
    for cut_number, cut in enumerate(cuts):
        split = get_train_test_split(num_data, target, cut)
        for number in range(tries):
            model_name_current = model_name(s, cut, number)
            history, model = model_train(split, batch_size, epochs) 
            
            model.save(model_name_current)
            histories[i][cut_number].append(history)

log_final(log_file, t)

Epoch 1/128


24/24 [==============================] - 1s 21ms/step - loss: 0.6883 - accuracy: 0.5468 - val_loss: 0.6823 - val_accuracy: 0.5648
Epoch 2/128
24/24 [==============================] - 0s 4ms/step - loss: 0.6764 - accuracy: 0.5818 - val_loss: 0.6754 - val_accuracy: 0.5724
Epoch 3/128
24/24 [==============================] - 0s 4ms/step - loss: 0.6657 - accuracy: 0.5855 - val_loss: 0.6657 - val_accuracy: 0.5701
Epoch 4/128
24/24 [==============================] - 0s 4ms/step - loss: 0.6553 - accuracy: 0.5896 - val_loss: 0.6569 - val_accuracy: 0.5881
Epoch 5/128
24/24 [==============================] - 0s 5ms/step - loss: 0.6460 - accuracy: 0.6039 - val_loss: 0.6500 - val_accuracy: 0.5975
Epoch 6/128
24/24 [==============================] - 0s 6ms/step - loss: 0.6373 - accuracy: 0.6134 - val_loss: 0.6414 - val_accuracy: 0.6162
Epoch 7/128
24/24 [==============================] - 0s 6ms/step - loss: 0.6297 - accuracy: 0.6245 - val_loss: 0.6336 - val_accuracy: 0.6188
Epoch 8/128
24/24 [=====